# Analisi de sentimientos noticias

**Autores: 
- Edgar Sebastian peña diaz
- leyder bolaños
- Breyner Andreit Rincon Quiroga
- Cristian Ballen Gamboa**\
Bogotá, Colombia \
Actualizado: 01 jul 2024 \
_Nota: Código ajustado para su ejecución en Google Colaboratory_\
Verificado con las siguientes versiones: Python 3.10.12, Tensorflow: 2.13.0, Keras: 2.13.1_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas openpyxl textblob nltk pysentimiento

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731

In [ ]:
# Importación de las librerías
import pandas as pd
from textblob import TextBlob
import nltk
import re
from nltk.corpus import stopwords
from pysentimiento import create_analyzer
import transformers

nltk.download('punkt')
nltk.download('stopwords')

transformers.logging.set_verbosity(transformers.logging.ERROR)

analyzer = create_analyzer(task="sentiment", lang="es")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [ ]:
def limpiar_texto(texto):
    if isinstance(texto, float):
        texto = ""
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar URLs
    texto = re.sub(r'http\S+', '', texto)
    # Eliminar menciones a usuarios de Twitter
    texto = re.sub(r'@\w+', '', texto)
    # Eliminar hashtags
    texto = re.sub(r'#\w+', '', texto)
    # Eliminar signos de puntuación y caracteres especiales
    texto = re.sub(r'[^\w\s]', '', texto)
    # Eliminar palabras vacías (stopwords)
    stop_words = set(stopwords.words('spanish'))
    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stop_words]
    texto = ' '.join(palabras)
    return texto

In [ ]:
def analizar_sentimiento(texto):
    resultado = analyzer.predict(texto)
    return resultado.output, resultado.probas

In [ ]:
# Cargue del documento
noticias = pd.read_excel('/content/drive/MyDrive/bigData/laboratorio final/Noticias2ver.xlsx')

# Aplicar la limpieza de texto a las columnas relevantes
noticias['info_limpio'] = noticias['info'].apply(limpiar_texto)
noticias['contenido_limpio'] = noticias['contenido'].apply(limpiar_texto)

In [ ]:
# Aplicar el análisis de sentimiento a las columnas limpias
noticias['sentimiento_info'] = noticias['info_limpio'].apply(lambda x: analizar_sentimiento(x)[0])
noticias['probabilidades_info'] = noticias['info_limpio'].apply(lambda x: analizar_sentimiento(x)[1])
noticias['sentimiento_contenido'] = noticias['contenido_limpio'].apply(lambda x: analizar_sentimiento(x)[0])
noticias['probabilidades_contenido'] = noticias['contenido_limpio'].apply(lambda x: analizar_sentimiento(x)[1])

In [ ]:
noticias.to_excel('/content/drive/MyDrive/bigData/laboratorio final/Noticias2ver_resultados.xlsx', index=False)

print("Análisis de sentimiento completado y resultados guardados en Noticias2ver_resultados.xlsx")

Análisis de sentimiento completado y resultados guardados en Noticias2ver_resultados.xlsx
